In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.common.exceptions import ElementNotInteractableException

In [ ]:
from time import sleep
from random import randint

import pandas as pd
import os

## **Crawl brand info**

In [ ]:
def get_brand_info(url):
    browser = webdriver.Chrome()
    browser.get(url)

    sleep(randint(3,6))

    name_brands = []
    link_brands = []

    
    list_brands = browser.find_elements(By.CLASS_NAME, 'list-brand')

    for b in list_brands:
        tag_link = b.find_elements(By.CLASS_NAME, 'list-brand__item.button__link')
        for t in tag_link:
            # Get link brand
            link = t.get_attribute('href')

            link_segments = link.split("/")[-1]
            name_brand = link_segments.split('.')[0]
            
            print(link)
            print(name_brand)

            link_brands.append(link)  
            name_brands.append(name_brand) 

    browser.close()
    return name_brands, link_brands

## **Function get Pruduct data**

In [ ]:
def get_product_info(url):

    browser = webdriver.Chrome()
    browser.get(url)
    sleep(30)

    try:
        # Wait for the "subscriber-popup box-banner" to appear
        popup_element = WebDriverWait(browser, 30).until(
            ec.presence_of_element_located((By.CLASS_NAME, "subscriber-popup.box-banner"))
        )

        # If the popup appears, click the "subscriber-form-submit button cancel-button"
        if popup_element.is_displayed():
            submit_button = browser.find_element(By.CLASS_NAME, "subscriber-form-submit.button.cancel-button")
            submit_button.click()
            sleep(randint(3, 6))

    except Exception as e:
        print(f"An error occurred: {e}")

    list_brand = browser.find_elements(By.CLASS_NAME, 'list-brand__item.button__link')
    names = []
    brand_ids = []
    links = []
    current_prices = []
    old_prices = []
    percent_discounts = []
    

    for i in range(1, len(list_brand)+1):
        
        # Click to a brand
        WebDriverWait(browser, randint(7, 9)).until(ec.element_to_be_clickable((By.XPATH, f'/html/body/div[1]/div/div/div[3]/div[2]/div/div[4]/div/div/a[{i}]'))).click()
                                                                                            # /html/body/div[1]/div/div/div[3]/div[2]/div/div[3]/div/div/a[{i}] mobile, tablet, watch and
                                                                                            # /html/body/div[1]/div/div/div[3]/div[2]/div/div[4]/div/div/a[{i}] laptop, Tivi

                                                                             
        sleep(randint(3, 5))

        
        show_more_btn = browser.find_elements(By.CLASS_NAME, 'button.btn-show-more.button__show-more-product')
        if len(show_more_btn) > 0:
            path_product_element = f'/html/body/div[1]/div/div/div[3]/div[2]/div/div[4]/div[5]/div/div[1]/div[x]'
            while True:
                try:
                    # click see more: if have button see more, and click until button 'see more' no longer appear
                    show_more_btn[0].click()
                    sleep(randint(2, 3))
                except Exception as e:
                    break
        
        else:
            path_product_element = f'/html/body/div[1]/div/div/div[3]/div[2]/div/div[3]/div[5]/div/div[1]/div[x]'



        items = browser.find_elements(By.CLASS_NAME, 'product-info-container.product-item')
                      
        for x in range(1, len(items)+1):          
            
            products = browser.find_elements(By.XPATH, path_product_element.replace('x', str(x)))
            for product in products:

                # get brand ID
                brand_id = str(i)
                print(brand_id)
                brand_ids.append(brand_id)

                # name product
                name = product.find_elements(By.CLASS_NAME, 'product__name')
                if len(name) > 0:
                    for n in name:
                        print(n.text)
                        names.append(n.text)

                else:
                    print('No name')
                    names.append('No name')

                # link product
                link = product.find_elements(By.CLASS_NAME, 'product__link.button__link')
                if len(link) > 0:
                    for l in link:
                        print(l.get_attribute('href'))
                        links.append(l.get_attribute('href'))
                else:
                    print('No link found')
                    links.append('No link found')

                # current prices
                current_price = product.find_elements(By.CLASS_NAME, 'product__price--show')
                if len(current_price) > 0:
                    for cp in current_price:
                        print(cp.text)
                        current_prices.append(cp.text)
                else:                    
                    print("Giá Liên Hệ")
                    current_prices.append("Giá Liên Hệ")

                # old_price
                old_price = product.find_elements(By.CLASS_NAME, 'product__price--through')
                if len(old_price) > 0:
                    for op in old_price:
                        print(op.text)
                        old_prices.append(op.text)
                else:
                    print("0")
                    old_prices.append("0")

                # percent discount
                percent_discount = product.find_elements(By.CLASS_NAME, 'product__price--percent')
                if len(percent_discount) > 0:
                    for pd in percent_discount:
                        print(pd.text)
                        percent_discounts.append(pd.text)
                else:
                    print("giảm 0%")
                    percent_discounts.append("giảm 0%")             

        browser.back()
        sleep(randint(3, 5))
    
    browser.close()
    return brand_ids, names, links, current_prices, old_prices, percent_discounts

## **Function build brand data frame**

In [ ]:
def build_brand_data_frame(name_brands, link_brands):

    brand = pd.DataFrame()

    brand['brand_name'] = name_brands
    brand['link'] = link_brands

    index = []
    for i in range(1, len(brand) + 1):
        index.append(str(i))

    brand['brand_id'] = index

    return brand

## **Function build Product data frame**

In [ ]:
def build_product_data_frame(brand_ids, names, links, current_prices, old_prices, percent_discounts):
    product = pd.DataFrame()

    product['product_name'] = names
    product['brand_id'] = brand_ids
    product['product_link'] = links
    product['current_price'] = current_prices
    product['original_price'] = old_prices
    product['percent_discount'] = percent_discounts

    return product

## **Laptop**

In [ ]:
name_brands, link_brands = \
get_brand_info('https://cellphones.com.vn/tivi.html')

In [ ]:
brand_ids, names, links, current_prices, old_prices, percent_discounts = \
get_product_info('https://cellphones.com.vn/tivi.html')

In [ ]:
print(len(brand_ids))
print(len(names))
print(len(links))
print(len(current_prices))
print(len(old_prices))

In [ ]:
print(len(name_brands))
print(len(link_brands))

In [ ]:
brand_data = build_brand_data_frame(name_brands, link_brands)
product_data = build_product_data_frame(brand_ids, names, links, current_prices, old_prices, percent_discounts)

In [ ]:
brand_data.head()

In [ ]:
product_data.head()

In [ ]:
current_dir = os.getcwd()
brand_data.to_csv(current_dir + '/brands/brand_tv.csv', index=False)
product_data.to_csv(current_dir + '/products/tv.csv', index=False)